In [1]:
import urllib.parse
import random
import oauthlib
from requests_oauthlib import OAuth1Session
import credentials_History
import twitter_settings  
import imp 
imp.reload(twitter_settings)
imp.reload(credentials_History)
from twitter_settings import *
import re
import tweepy
import mysql.connector
import pandas as pd
import numpy as np
#from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import datetime
from datetime import date, timedelta
import random
import time
from datetime import datetime, timezone
import pytz

In [2]:
def clean_tweet(tweet): 
     
    ##Use sumple regex statemnents to clean tweet text by removing links and special characters
    
    tweet_text= ' '.join(re.sub("(RT @[\w_]+:)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
    tweet_text = re.sub(r'(.)\1{2,}',r'\1\1',tweet_text)
    return tweet_text

def deEmojify_tweet(text):
    
    ##Strip all non-ASCII characters to remove emoji characters
    
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="*******",
    database="TwitterDB",
    auth_plugin='mysql_native_password',
    charset='utf8'
    
)
if mydb.is_connected():
    mycursor = mydb.cursor()
    mycursor.execute("SELECT COUNT(*) FROM information_schema.tables where table_name='{0}'".
                     format(twitter_settings.HISTORY_TABLE_NAME))
if mycursor.fetchone()[0] != 1:
    mycursor.execute("CREATE TABLE {} ({})".format(twitter_settings.HISTORY_TABLE_NAME, twitter_settings.HISTORY_TABLE_ATTRIBUTES))
    mydb.commit()
    mycursor.close()

In [7]:
def save_tweets(status,identifier):
    try:
        print("Inside tweet function")
        #print(len(tweets_list))
        
        print("Tweet started")

        print('Retweet status: {0}'.format(status["retweeted"]))
        print('Original tweet: {0}'.format(status["text"]))

        if status["retweeted"]:
            ## for now avoid retweeted info
            return True

        #Extract attributes of each tweet
        id_str=status["id_str"]
        created_at=status["created_at"]
        print("created at: {0}".format(created_at))
        created_at=datetime.strptime(status["created_at"], '%a %b %d %H:%M:%S %z %Y').replace(
            tzinfo=timezone.utc).astimezone(pytz.timezone('US/Eastern')).strftime('%Y-%m-%d %H:%M:%S')
        print("created at: {0}".format(created_at))
        original_text=status["text"]
        text=deEmojify_tweet(status["text"])
        text=clean_tweet(text)

        print('Clean tweet: {0}'.format(text))

        ## use Vader to check sentiment of text
        ########################################
        analyser = SentimentIntensityAnalyzer()

        score = analyser.polarity_scores(status["text"])
        print("{:-<40} {}".format(text, str(score)))

        if score['compound']>=0.05: #positive
            polarity = 1
        elif (score['compound']>-0.05) & (score['compound']<0.05): #neutral
            polarity = 0
        elif score['compound']<=-0.05: #negative
            polarity = -1

        compound_score=score['compound']
        positive_score=score['pos']
        negative_score=score['neg']
        neutral_score=score['neu']

        print('polarity: {0}'.format(polarity))

        ## set subjectivity as 0 for Vader
        subjectivity = 0
        ###########################################

        ## use TextBlob to check sentiment of text
        ########################################
    #         sentiment=TextBlob(text).sentiment
    #         polarity=sentiment.polarity
    #         subjectivity=sentiment.subjectivity
        ######################################



        
       
        #usermentions_list = json.dump(user_mentions)
        user_id = status["user"]["id_str"]
        user_name = status["user"]["name"]
        user_screen_name = status["user"]["screen_name"]
        print('userId: {0}, userName: {1}, userScreenName: {2}'.format(user_id,user_name,user_screen_name))
        user_created_at = status["user"]["created_at"]
        user_location=deEmojify_tweet(status["user"]["location"])
        user_description=deEmojify_tweet(status["user"]["description"])

        user_friends_count = status["user"]["friends_count"]
        user_followers_count=status["user"]["followers_count"]
        user_following = status["user"]["following"]
        print('user follower count: {0}, userFollowing: {1}'.format(user_followers_count,user_following))
        print('user friends count: {0}'.format(user_friends_count))

        #------------------------------------------
        tweetFound=False
        track_val = ''
        tweet_found_count = 0

        netflix_check_list = ['Netflix','netflix']
        netflix_check_set = set(netflix_check_list)
#         appletv_check_list = ['AppleTV','Apple TV','appletv','apple tv']
#         appletv_check_set = set(appletv_check_list)
#         disney_check_list = ['DisneyPlus','Disney+','disneyplus','disney+','disney','Disney','Disneyplus']
#         disney_check_set = set(disney_check_list)
#         amazon_check_list = ['Amazon Prime Video','amazon prime video','AmazonPrimeVideo',
#                      'amazonprimevideo','AmazonPrime Video','Amazon Prime','amazon prime','amazon','prime',
#                     'Amazon','Prime']
#         amazon_check_set = set(amazon_check_list)
#     #         hbo_check_list = ['HBOMax','HBO Max','hbomax']
#     #         hbo_check_set = set(hbo_check_list)

        text_set = set(text.split())
        print('text set: {0}'.format(text_set))
#         if amazon_check_set.intersection(text_set):
#             track_val = 'AmazonPrime'
#             tweetFound=True
#             tweet_found_count = tweet_found_count + 1
#         if appletv_check_set.intersection(text_set):
#             track_val = 'AppleTV'
#             tweetFound=True
#             tweet_found_count = tweet_found_count + 1
        print('Identifier: {0}'.format(identifier))
#         if (identifier=='Disney') and (disney_check_set.intersection(text_set)):
#             track_val = 'DisneyPlus'
#             tweetFound=True
#             tweet_found_count = tweet_found_count + 1
        if (identifier=='Netflix') and (netflix_check_set.intersection(text_set)):
            track_val = 'Netflix'
            tweetFound=True
            tweet_found_count = tweet_found_count + 1

        if (tweet_found_count > 1):
            tweetFound = False

        print("tweet found: {0}".format(tweetFound))
        print("Lets begin to record! All the best :)")

        longitude=None
        latitude=None
#         if status["coordinates"]:
#             longitude=status["coordinates"]["coordinates"][0]
#             latitude=status["coordinates"]["coordinates"][1]

            
        print('latitude: {0}  logitude: {1}'.format(latitude,longitude))
        
        retweet_count=status["retweet_count"]
        favorite_count=status["favorite_count"]

        print('retweet count: {0}'.format(retweet_count))
        print('faviourite count: {0}'.format(favorite_count))
        
        hashtags = []   #make an empty list

#         if status["entities"]!=null:
#             print("hashtags: {0}".format(status["entities"]))

        for hashtag in status["entities"]["hashtags"]:    #iterate over the list
            hashtags.append(hashtag["text"])             #append each hashtag to 'hashtags'

        print('hashtags: {0}'.format(str(hashtags)))

        if tweetFound:
            if mydb.is_connected():
                mycursor=mydb.cursor()

                sql = "INSERT IGNORE INTO {} (id_str, created_at, track ,text, original_text, polarity, compound_score, negative_score, positive_score, neutral_score,subjectivity, hashtags,\
                user_created_at, user_id, user_name, user_screen_name,\
                user_location, user_description, user_followers_count, user_friends_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(twitter_settings.HISTORY_TABLE_NAME)
                val = (id_str, created_at, track_val, text, original_text, polarity, compound_score, negative_score, positive_score, neutral_score ,subjectivity, str(hashtags), user_created_at, user_id, user_name, user_screen_name, user_location, \
                    user_description, user_followers_count, user_friends_count, longitude, latitude, retweet_count, favorite_count)
                mycursor.execute(sql, val)
                mydb.commit()
                mycursor.close()
                
                print("Insert done!")
    except:   
        return False

In [26]:

class GetTweets:
    def parse_config(self):
        config = {}
      # from file args
        if os.path.exists('data/config.json'):
            with open('data/config.json') as f:
                config.update(json.load(f))
      # should have something now
        return config

    def oauth_req(self, url, http_method="GET", post_body=None,
                  http_headers=None):
        #config = self.parse_config()
        twitter = OAuth1Session(credentials_History.API_KEY,client_secret=credentials_History.API_SECRET_KEY,resource_owner_key=credentials_History.ACCESS_TOKEN,resource_owner_secret=credentials_History.ACCESS_TOKEN_SECRET)
        resp = twitter.request(http_method,url)
        return resp
    
    #start getTwitterData
    def getData(self, keyword, params = {}):
        maxTweets = 1000
        url = 'https://api.twitter.com/1.1/search/tweets.json?'    
        data = {'q': keyword, 'lang': 'en', 'result_type': 'recent', 'count': maxTweets, 'include_entities': 1}
        #Add if additional params are passed
        if params:
            for key, value in params.items():
                data[key] = value
        
        url += urllib.parse.urlencode(data)
        response = self.oauth_req(url)          
        jsonData = json.loads(response.content.decode('UTF-8'))

        if 'errors' in jsonData:
            print("API Error")
            print(jsonData['errors'])
        else:
            return jsonData
    #end    

    def getTwitterData(self, symbol,identifier):
        #tomorrow = datetime.datetime.now()+timedelta(days=+1)
        since_date = datetime.now() - timedelta(days=10)
        until_date = datetime.now() - timedelta(days=3)
        print(until_date)
        until_now = datetime.now() 
        until_future = until_date + timedelta(days=1)
        print(until_now)
        #params = {'since': since_date.strftime("%Y-%m-%d"), 'until': until_date.strftime("%Y-%m-%d")}
        params = {'since': until_date.strftime("%Y-%m-%d"), 'until': until_future.strftime("%Y-%m-%d")}
        jsonData = self.getData(symbol, params)
        tweets = []
        for item in jsonData['statuses']:
            myData = {}
            myData["created_at"] = item["created_at"]
            myData["text"] = item["text"]
            myData["user"] = item["user"]["screen_name"]
            save_tweets(item,identifier)
            tweets.append(myData)
        print(jsonData)
        return tweets
 
 
if __name__ == "__main__":
    a  = GetTweets()
    print(a.getTwitterData('Netflix or netflix','Netflix'))
    #print(a.getTwitterData('DisneyPlus OR Disney+ OR disneyplus OR disney+','Disney'))

2019-11-30 13:33:57.277101
2019-12-03 13:33:57.277213
Inside tweet function
Tweet started
Retweet status: False
Original tweet: RT @AimanIAbdullah: I Netflix when I have lunch at work.

For many of us, our jobs demand interaction with people/many people whether we li…
created at: Sat Nov 30 23:58:49 +0000 2019
created at: 2019-11-30 18:58:49
Clean tweet: I Netflix when I have lunch at work For many of us our jobs demand interaction with people many people whether we li
I Netflix when I have lunch at work For many of us our jobs demand interaction with people many people whether we li {'neg': 0.067, 'neu': 0.933, 'pos': 0.0, 'compound': -0.128}
polarity: -1
userId: 495700546, userName: Golden Recipe, userScreenName: AisyahSyarip
user follower count: 422, userFollowing: False
user friends count: 1130
text set: {'when', 'Netflix', 'li', 'with', 'lunch', 'I', 'work', 'of', 'our', 'For', 'us', 'jobs', 'we', 'at', 'many', 'have', 'demand', 'whether', 'people', 'interaction'}
Identifier: Netf

My local newspaper from yesterday we will NOT go unheard or go down WITHOUT a fight {'neg': 0.0, 'neu': 0.856, 'pos': 0.144, 'compound': 0.5191}
polarity: 1
userId: 1146422223290413056, userName: Anil zineb, userScreenName: AnilZineb
user follower count: 20, userFollowing: False
user friends count: 30
text set: {'we', 'NOT', 'newspaper', 'My', 'local', 'WITHOUT', 'yesterday', 'or', 'down', 'from', 'go', 'unheard', 'will', 'fight', 'a'}
Identifier: Netflix
tweet found: False
Lets begin to record! All the best :)
latitude: None  logitude: None
retweet count: 354
faviourite count: 0
hashtags: []
Inside tweet function
Tweet started
Retweet status: False
Original tweet: At least someone keeps their mind not like other...@netflix or @CBC .... #renewannewithane #SaveAnneWithAnnE… https://t.co/xsB5IZbj7I
created at: Sat Nov 30 23:48:38 +0000 2019
created at: 2019-11-30 18:48:38
Clean tweet: At least someone keeps their mind not like other or renewannewithane SaveAnneWithAnnE
At least someone k

faviourite count: 4
hashtags: []
Inside tweet function
Tweet started
Retweet status: False
Original tweet: RT @iHasMagic: The actual title would prob be "Gung-ho! Preyour 10" or something LOL, but crunchyroll's subtitles decided to also parody pr…
created at: Sat Nov 30 23:38:00 +0000 2019
created at: 2019-11-30 18:38:00
Clean tweet: The actual title would prob be Gung ho Preyour 10 or something LOL but crunchyroll s subtitles decided to also parody pr
The actual title would prob be Gung ho Preyour 10 or something LOL but crunchyroll s subtitles decided to also parody pr {'neg': 0.0, 'neu': 0.891, 'pos': 0.109, 'compound': 0.3733}
polarity: 1
userId: 905127726616526848, userName: cap, userScreenName: planthomo
user follower count: 65, userFollowing: False
user friends count: 89
text set: {'would', '10', 'ho', 'Gung', 's', 'be', 'also', 'decided', 'actual', 'something', 'but', 'subtitles', 'prob', 'LOL', 'title', 'or', 'to', 'parody', 'crunchyroll', 'Preyour', 'The', 'pr'}
Identifier: N

Retweet status: False
Original tweet: RT @thehorsegirl15: Amenadiel: Lucifer, I'm not angry with u that u somehow got your wings back, or even that u cut them off again, as foul…
created at: Sat Nov 30 23:29:35 +0000 2019
created at: 2019-11-30 18:29:35
Clean tweet: Amenadiel Lucifer I m not angry with u that u somehow got your wings back or even that u cut them off again as foul
Amenadiel Lucifer I m not angry with u that u somehow got your wings back or even that u cut them off again as foul {'neg': 0.081, 'neu': 0.814, 'pos': 0.105, 'compound': 0.1536}
polarity: 1
userId: 458535820, userName: Karen Chan, userScreenName: karenchan28
user follower count: 101, userFollowing: False
user friends count: 505
text set: {'foul', 'got', 'as', 'u', 'with', 'angry', 'again', 'wings', 'cut', 'Lucifer', 'them', 'I', 'm', 'or', 'not', 'Amenadiel', 'even', 'back', 'off', 'somehow', 'that', 'your'}
Identifier: Netflix
tweet found: False
Lets begin to record! All the best :)
latitude: None  logitude:

## Extra

In [223]:
# def big_func():
    
#     def trial_score_check(text):
    
#         analyser11 = SentimentIntensityAnalyzer()
#         tt_score = analyser11.polarity_scores(text)
#         return tt_score
#     fd_vader = pd.DataFrame(data=['good','bad','fair','plain','so','night'], columns = ["Word"]).drop([0]).reindex()
#     fd_vader['Compound_Score'] = fd_vader['Word'].apply(trial_score_check)
#     print(fd_vader['Compound_Score'])
    
# big_func()

In [224]:
# def trial_score_check(text):
    
#         analyser11 = SentimentIntensityAnalyzer()
#         tt_score = analyser11.polarity_scores(text)
#         return tt_score['compound']

# fd_vader = pd.DataFrame(data=['good','bad','fair','plain','so','night'], columns = ["Word"]).drop([0]).reindex()
# fd_vader['Compound_Score'] = fd_vader['Word'].apply(trial_score_check)
# fd_vader['Compound_Score']#[1]